# Generating Word Embeddings - Lab

## Introduction

In this lab, we'll learn how to generate our own word embeddings by training our own Word2Vec model, and also by building embedding layers right into our Deep Neural Networks!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, we'll start by creating our own word embeddings by making use of the Word2Vec Model. Then, we'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in our model. 

The easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which we can use to begin training immediately. For this lab, we'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.  In this lab, we'll learn how to train a Word2Vec model on the text data to generate word embeddings for them. In the next lab, we'll then use the vectors created by our Word2Vec model to effectively train a classifier to predict the category of news given the headline and description of each article. In this lab, we won't do any classification, although we will learn how to train a Word2Vec model and explore the relationships between different word vectors in our embedding!

Run the cell below to import everything we'll need for this lab. 

In [3]:
!pip install gensim

    100% |████████████████████████████████| 24.7MB 493kB/s eta 0:00:01   19% |██████▏                         | 4.7MB 30.2MB/s eta 0:00:01    36% |███████████▋                    | 8.9MB 17.7MB/s eta 0:00:01    46% |███████████████                 | 11.6MB 13.6MB/s eta 0:00:01    57% |██████████████████▍             | 14.2MB 17.8MB/s eta 0:00:01
    100% |████████████████████████████████| 40kB 5.8MB/s ta 0:00:011
    100% |████████████████████████████████| 1.4MB 8.8MB/s eta 0:00:01    21% |███████                         | 296kB 12.2MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 17.5MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 14.1MB/s ta 0:00:01
    99% |████████████████████████████████| 5.3MB 26.2MB/s eta 0:00:01   4% |█▍                              | 235kB 21.0MB/s eta 0:00:01    100% |████████████████████████████████| 5.3MB 5.3MB/s 
    100% |████████████████████████████████| 552kB 11.2MB/s ta 0:00:01
  Stored in directory: /Users/natalia/Li

In [4]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, we'll import the data. You'll find the data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to also include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what our data looks like. 

In [6]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since we're working with text data, we'll still need to do some basic preprocessing and tokenize our data. You'll notice from the sample of the data above that two different columns contain text data--`headline` and `short_description`. The more text data our Word2Vec model has, the better it will perform. Therefore, we'll want to combine the two columns before tokenizing each comment and training our Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [26]:
df['combined_text'] = (df.headline + ' ' + df.short_description).str.lower()
df['combined_text'][0]

'there were 2 mass shootings in texas last week, but only 1 on tv she left her husband. he killed their children. just another day in america.'

In [27]:
data = df['combined_text'].map(word_tokenize)    #tokenizes every single sentence in the set / row

Let's inspect the first 5 items in `data` to see how everything looks. 

In [28]:
data[:5]

0    [there, were, 2, mass, shootings, in, texas, l...
1    [will, smith, joins, diplo, and, nicky, jam, f...
2    [hugh, grant, marries, for, the, first, time, ...
3    [jim, carrey, blasts, 'castrato, ', adam, schi...
4    [julianna, margulies, uses, donald, trump, poo...
Name: combined_text, dtype: object

In [ ]:
# Noise elimination is required: lowercase, numbers, apostrophes, joint-words...

You'll notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Recall from our previous lesson on how Word2Vec works that we can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If our window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example we saw).

Now that we've prepared our data, let's train our model and explore a bit!

## Training the Model

We'll start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [47]:
model = Word2Vec(data, size=50, window=5, min_count=1, workers=4)

Now, that we've created our Word2Vec model, we still need to train it on our model. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which we can find in `model.corpus_count`. 
    * The number of `epochs` we want to train for, which we'll set to `10`

In [48]:
model.train(data, total_examples=model.corpus_count, epochs=10)
#total_examples : int Count of sentences.
#epochs : int Number of iterations (epochs) over the corpus.
#returns dimensionality

(52848298, 67317580)

Great! We now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which we'll find stored inside of `model.wv`. For simplicity's sake, let's go ahead and store this inside of the variable `wv` in order to save ourselves some keystrokes down the line. 

In [49]:
wv = model.wv

## Examining Our Word Vectors

Now that we have a trained Word2Vec model, let's go ahead and explore the relationships between some of the words in our corpus! 

One cool thing we can use Word2Vec for is to get the most similar words to a given word. We can do this passing in the word to `wv.most_similar()`. 

In the cell below, let's try getting the most similar word to `'Texas'`.

In [50]:
wv.most_similar('texas')

[('florida', 0.8840732574462891),
 ('ohio', 0.8781334161758423),
 ('arkansas', 0.8696154356002808),
 ('michigan', 0.8677798509597778),
 ('pennsylvania', 0.8616589307785034),
 ('connecticut', 0.8580466508865356),
 ('missouri', 0.8576474189758301),
 ('illinois', 0.8555783033370972),
 ('louisiana', 0.854843258857727),
 ('oklahoma', 0.8535414338111877)]

Interesting! All of the most similar words are also states. 

We can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [51]:
wv.most_similar(negative='texas')

[('coronae', 0.5339177250862122),
 ('parent/grandparent', 0.5209290385246277),
 ('much-vaunted', 0.4931260347366333),
 ('pusha', 0.4879226088523865),
 ('huffine', 0.4755060076713562),
 ('pares', 0.44576549530029297),
 ('boastfully', 0.42520540952682495),
 ('p85', 0.42021414637565613),
 ('adulterers', 0.4070592522621155),
 ('gatsby-style', 0.40593674778938293)]

These seem like just noise. This is because of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with our given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

We can also get the vector for a given word by passing in the word as if we were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [52]:
wv['texas']

array([-2.9328623 , -2.2183094 ,  0.09333005, -0.61504656, -1.072538  ,
       -0.8850166 , -1.2659628 , -0.09701044,  1.4564854 ,  0.38763624,
        0.5847864 , -0.19685228,  0.46645224,  0.0980379 ,  0.40323928,
        1.723101  , -1.3094556 ,  1.9348423 ,  3.0623302 ,  0.01417603,
       -0.11053841,  0.31758365,  1.6010435 ,  0.4553911 ,  1.2575455 ,
        2.3134217 , -0.7346281 , -1.7192566 , -0.9735134 ,  0.4182475 ,
        2.0234938 ,  3.23633   ,  0.951954  ,  4.080785  ,  0.12739234,
       -1.3047622 ,  1.5820266 ,  0.8359725 , -0.02095622, -0.58303785,
        0.02683572,  2.64159   ,  2.8941944 , -2.5183473 ,  1.5916736 ,
        0.17607449, -0.89509404,  0.5224622 , -2.1962218 , -0.4598483 ],
      dtype=float32)

In [53]:
wv['texas'].shape   #from the size parameter 

(50,)

Let's get all of the word vectors from the object at once. We can find these inside of `wv.vectors`.  Do this now in the cell below.  

In [54]:
wv.vectors

array([[ 0.497678  , -1.2724055 ,  0.34506688, ...,  0.2755453 ,
         0.11933431, -0.13075618],
       [ 2.2618303 , -0.8974778 , -0.9891427 , ..., -1.7945607 ,
         1.3301427 ,  0.96985525],
       [ 1.5792447 , -1.2740972 , -1.0565239 , ..., -1.4586254 ,
         1.7669201 ,  1.8121969 ],
       ...,
       [-0.02550895, -0.04860802,  0.0778411 , ..., -0.00820009,
        -0.00756679, -0.07384065],
       [ 0.07047282,  0.01232571, -0.17574292, ...,  0.03121552,
        -0.06777763, -0.02071894],
       [-0.01574019,  0.03252747,  0.08587164, ...,  0.0090966 ,
         0.03348228, -0.0503365 ]], dtype=float32)

As a final exercise, let's try recreating the _'king' - 'man' + 'woman' = 'queen'_ example we've seen before. We can do this by using the `most_similar` function and putting the things we want added together inside of an array passed to the `positive` parameter, and the things we want subtracted as an array passed to the the `negative` parameter. 

Do this now in the cell below. 

In [69]:
wv.most_similar(positive=['king', 'woman'], negative=['man'], topn=30)

[('nelson', 0.6613090634346008),
 ('r.r', 0.6581754684448242),
 ('wiseman', 0.6559332609176636),
 ('martin', 0.6525302529335022),
 ('scorsese', 0.639789879322052),
 ('sherman', 0.6342746615409851),
 ('prize-winning', 0.6325646638870239),
 ('regina', 0.6229173541069031),
 ('luther', 0.6220930814743042),
 ("l'engle", 0.6156068444252014),
 ('lincoln', 0.6135976910591125),
 ('saint', 0.6078938245773315),
 ('joseph', 0.6064093112945557),
 ('poet', 0.6062799096107483),
 ('frederick', 0.603554904460907),
 ('jackie', 0.6011104583740234),
 ('mujahid', 0.5972909331321716),
 ('farrah', 0.5932519435882568),
 ('bernard', 0.5931751132011414),
 ('romero', 0.5924740433692932),
 ('sameera', 0.5923437476158142),
 ('carol', 0.5914967060089111),
 ('jewel', 0.5899664759635925),
 ("'sontag", 0.5897639989852905),
 ('maison', 0.5877974629402161),
 ('queen', 0.5849304795265198),
 ('barry', 0.5836139917373657),
 ('mandela', 0.583180844783783),
 ('leonard', 0.5827056169509888),
 ('renisha', 0.5822892189025879)]

As we can see from the output above, our model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As we can see from the word in first place, 'reminiscent', our model is far from perfect. This is likely because we didn't give it too much training, or training data. However, for the small amount of training data it was given, the model still performs remarkably well! 

We'll see in the next lab that from a practical standpoint, one of the best things we can do for performance is to start by loading in the weights from an open-sourced model that has been trained for a very long time on a massive amount of data, such as the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless our text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, we learned how to train and use a Word2Vec model to created vectorized word embeddings!